# Problema de designação de pessoas **I** para instrumentos **J** e vozes **L** no dia **K**
---

##  i - Conjunto de pessoas
##  J - Conjunto de instrumentos
##  k - Conjunto de dias
##  l - Conjunto de vozes

---

* Mij = Matriz Binária - Pessoa I toca Instrumento J

* Sil = Matriz Binária - Pessoa I toca canta na posição L

* PMj = Peso de cada instrumento J
* PSl = Peso de cada voz L

* Dk = Peso de cada dia K

### Solução
* Xijk - Matriz Binária no R3
* Yilk - Matriz Binária no R3
* Zik -  Matriz binária no R2

* **Se Xijk = 1 => A pessoa "i" tocará o instrumento 'j' no dia 'k'**

* Se Yijl = 1 => A pessoa "i" cantará na posição 'l' no dia 'k'

* **Se Xijk = 0 => A pessoa "i" não tocará o instrumento 'j' no dia 'k'**

* Se Yijl = 0 => A pessoa "i" não cantará na posição 'l' no dia 'k'

* **Somatório em i de Xijk <= 1 -  Um instrumento 'j' só pode ser tocado por uma pessoa 'i' em um dia 'k'**

* Somatório em i de Yilk <= 1 - Uma voz 'l' só pode ser cantada por uma pessoa 'i' em um dia 'k'

* **Somatório em 'j' de Xijk <= 1 - A pessoa "i" tocará somente um instrumento instrumento 'j' no dia 'k'**

* Somatório em 'l' de Yilk <= 1 - A pessoa "i" cantará somente em uma posição "l" no dia "k"

* **Zik - Pessoa i participa no dia k**
* **Zik = 0 se Xijk e yilk = 0**
* **Zik = 1 se Xijk ou yilk = 1**


* **Zik + Zi(k+1) + Zi(k+2) + Zi(k+4) <= 2 para todo i - A pessoa "i" não participará mais do que dois dias seguidos.**


* **Xijk = 0 se PMij = 0**
* Yilk = 0 se PSil = 0


* Maximize Xijk * Mij * PMj * Dk + Yilk * Sil * PSl * Dk

In [7]:
import pandas as pd

file_name = 'Dados Louvor Shalom.xlsx'

sheets_dict = pd.read_excel(file_name, sheet_name=['Instrumento', 'Peso dos Instrumentos', 'Vozes', 'Peso das Vozes', 'Peso dos Dias','Disponibilidade', 'Dias por Mês'])

# Acesse cada sheet pelo nome
#Dias por Mês
P = sheets_dict['Instrumento'].set_index('Pessoas')
V = sheets_dict['Vozes'].set_index('Pessoas')

L = sheets_dict['Peso dos Instrumentos'].set_index('Instrumentos').squeeze()
LV = sheets_dict['Peso das Vozes'].set_index('Vozes').squeeze()
D = sheets_dict['Peso dos Dias'].set_index('Dias')

Disp = sheets_dict['Disponibilidade'].set_index('Dias')
DPM = sheets_dict['Dias por Mês'].set_index('Pessoas')

# Convertendo os índices para o tipo datetime
D.index = pd.to_datetime(D.index)
Disp.index = pd.to_datetime(Disp.index)

# Formatando os índices para o formato desejado
D.index = D.index.strftime('%d_%m_%Y')
Disp.index = Disp.index.strftime('%d_%m_%Y')

from pulp import *

# Definir o problema
prob = LpProblem("Alocacao_de_Musicos", LpMaximize)

# Variáveis de decisão
X = LpVariable.dicts("X", (P.index, P.columns, D.index), cat='Binary')

# Função objetivo
prob += lpSum([X[i][j][k] * Disp[i][k] * P.loc[i, j] * L[j] * D['Peso'][k] for i in P.index for j in P.columns for k in D.index])

# Restrições
for j in P.columns:
    for k in D.index:
        prob += lpSum([X[i][j][k] for i in P.index]) <= 1

for i in P.index:
    for k in D.index:
        prob += lpSum([X[i][j][k] for j in P.columns]) <= 1

for i in P.index:
    for k in range(len(D.index) - 3):
        prob += lpSum([X[i][j][D.index[k]] + X[i][j][D.index[k+1]] + X[i][j][D.index[k+2]] + X[i][j][D.index[k+2]] for j in P.columns]) <= DPM['Quantidade'][i]

# Cada pessoa precisa tocar pelo menos uma vez no mês
# for i in P.index:
#     for k in range(len(D.index) - 3):
#         for j in P.columns:
#             if P.loc[i, j] == 1:
#                 prob += lpSum([X[i][j][D.index[k]] + X[i][j][D.index[k+1]] + X[i][j][D.index[k+2]] + X[i][j][D.index[k+2]]]) >= 1

for i in P.index:
    for j in P.columns:
        for k in D.index:
            if P.loc[i, j] == 0:
                prob += X[i][j][k] == 0

# Resolver o problema
prob.solve()

from IPython.display import clear_output

clear_output()

# Imprimir o status da solução
print("Status:", LpStatus[prob.status])

# Imprimir o valor da função objetivo
print("Valor da função objetivo =", value(prob.objective))

# Dicionário para armazenar os resultados
resultados = {}

# Iterar sobre cada variável de decisão
for v in prob.variables():
    if v.varValue != 0:
        # Extrair o nome da pessoa, o instrumento e o dia
        _, pessoa, instrumento, dia = v.name.split("_", 3)

        # Se o dia ainda não está no dicionário, adicione-o
        if dia not in resultados:
            resultados[dia] = {}

        # Adicione a pessoa e o instrumento ao dia correspondente
        resultados[dia][instrumento] = pessoa

df = pd.DataFrame(columns=P.columns, index=D.index)
escala_instrumental = pd.DataFrame(0, columns=P.index, index=D.index)

# Preencher o DataFrame com os resultados
for dia in resultados:
    for instrumento in resultados[dia]:
        row_index = dia
        column_name = resultados[dia][instrumento]
        escala_instrumental.loc[row_index, column_name] = 1
        df.loc[dia, instrumento] = resultados[dia][instrumento]

# # Substituir NaN por ''
df = df.fillna('')

# Convertendo os índices para o tipo datetime
df.index = pd.to_datetime(df.index, format='%d_%m_%Y')

# Formatando os índices para o formato desejado e criando um novo índice
formatted_index = df.index.strftime('%d/%m/%Y')

# Atribuindo o novo índice formatado ao DataFrame
df.index = formatted_index
df

Status: Optimal
Valor da função objetivo = 1326.0


,Bateria,Baixo,Teclado,Violão,Guitarra
Dias,,,,,
14/01/2024,Lucas,Daniel,Bernardo,Ricardo,Osvaldo
21/01/2024,Henrique,Ricardo,Mauri,Daniel,Osvaldo
28/01/2024,Marcela,Gabriel,Vinicius,Henrique,Osvaldo
04/02/2024,Lucas,Rafael,Bernardo,Osvaldo,Enzo
11/02/2024,Henrique,Ricardo,Vinicius,Osvaldo,Daniel
18/02/2024,Marcela,Gabriel,Henrique,Ricardo,Osvaldo
25/02/2024,Lucas,Enzo,Vinicius,Rafael,Osvaldo
03/03/2024,Henrique,Daniel,Vinicius,Ricardo,Osvaldo
10/03/2024,Marcela,Gabriel,Bernardo,Ricardo,Osvaldo


In [9]:
disponibilidade_parcial = pd.DataFrame(0, columns=P.index, index=D.index)

# Definir o problema
prob_disp = LpProblem("Disponibilidade Pessoal", LpMaximize)

# Variáveis de decisão
X = LpVariable.dicts("X", (escala_instrumental.index, escala_instrumental.columns), cat='Binary')

# Função objetivo
prob_disp += lpSum([X[date][person] for date in escala_instrumental.index for person in escala_instrumental.columns])

# Restrições para garantir que a soma de qualquer sequência de 4 dias consecutivos seja menor ou igual a DPM['Quantidade'][i]
for person in escala_instrumental.columns:
    for k in range(len(escala_instrumental.index) - 3):
        prob_disp += lpSum([X[escala_instrumental.index[i]][person] for i in range(k, k + 4)]) <= DPM.loc[person, 'Quantidade'], f"Constraint_{person}_{k}"

# Resolver o problema
prob_disp.solve()

# Atribuir os valores das variáveis de decisão ao DataFrame disponibilidade
for date in disponibilidade_parcial.index:
    for person in disponibilidade_parcial.columns:
        disponibilidade_parcial.loc[date, person] = X[date][person].varValue

for column in escala_instrumental.columns:
    if (escala_instrumental[column] == 0).all():
        disponibilidade_parcial[column] = 1

clear_output()

disponibilidade = pd.DataFrame(0, columns=P.index, index=D.index)
for i in Disp.index:
    for j in Disp.columns:
        # Verificar se o valor em N é igual ao valor em M
        if Disp.loc[i, j] == disponibilidade_parcial.loc[i, j]:
            # Se for igual, definir Xij como 1
            disponibilidade.loc[i, j] = 1

disponibilidade

Pessoas,Daniel,Enzo,Gabriel,Henrique,Lucas,Marcela,Mauri,Osvaldo,Ricardo,Vinicius,Israel,Everlyn,Dani,Júlia,Rafael,Tassi,Bernardo,Ariane
Dias,,,,,,,,,,,,,,,,,,
14_01_2024,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
21_01_2024,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1
28_01_2024,0,0,0,1,0,0,0,1,1,1,1,1,1,1,0,1,0,1
04_02_2024,0,0,0,0,0,0,0,1,0,0,1,1,1,1,0,1,0,1
11_02_2024,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
18_02_2024,0,1,1,1,1,0,0,1,1,1,1,1,1,1,0,1,0,1
25_02_2024,1,0,0,1,0,1,0,1,0,0,1,1,1,1,0,1,0,1
03_03_2024,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1
10_03_2024,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [11]:
from pulp import *

days = 2

# Definir o problema
prob_v = LpProblem("Alocacao_de_Musicos", LpMaximize)

# Variáveis de decisão
Y = LpVariable.dicts("Y", (V.index, V.columns, D.index), cat='Binary')

# Função objetivo
prob_v += lpSum([Y[i][j][k] * disponibilidade[i][k]* V.loc[i, j] * LV[j] * D['Peso'][k] for i in V.index for j in V.columns for k in D.index])

# Restrições

#Cada pessoa pode tocar apenas uma voz por dia
for j in V.columns: #Para cada voz
    for k in D.index: #Para cada Dia
        prob_v += lpSum([Y[i][j][k] for i in V.index]) <= 1 #Somatório de Y[i][j][k] para cada pessoa é menor ou igual a 1 

for i in V.index:  #Para cada voz pessoa
    for k in D.index:
        prob_v += lpSum([Y[i][j][k] for j in V.columns]) <= 1

for i in V.index:
    for k in range(len(D.index) - 3):
        prob_v += lpSum([Y[i][j][D.index[k]] + Y[i][j][D.index[k+1]] + Y[i][j][D.index[k+2]] + Y[i][j][D.index[k+2]] for j in V.columns]) <= DPM['Quantidade'][i]

for i in V.index:
    for j in V.columns:
        for k in D.index:
            if V.loc[i, j] == 0:
                prob_v += Y[i][j][k] == 0

#Resolver o problema
prob_v.solve()



Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/hmbarros/.local/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/bf1695898bf64340892311739c4672b4-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/bf1695898bf64340892311739c4672b4-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 978 COLUMNS
At line 7481 RHS
At line 8455 BOUNDS
At line 9392 ENDATA
Problem MODEL has 973 rows, 936 columns and 4539 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 555 - 0.00 seconds
Cgl0002I 507 variables fixed
Cgl0004I processed model has 75 rows, 77 columns (77 integer (77 of which binary)) and 290 elements
Cutoff increment increased from 1e-05 to 4.9999
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of -555
Cbc0038I Before mini branch and bound, 77 integers at bound fixed and 0 c

1

In [12]:
# Imprimir o status da solução
print("Status:", LpStatus[prob_v.status])

# Imprimir o valor da função objetivo
print("Valor da função objetivo =", value(prob_v.objective))

# Dicionário para armazenar os resultados_v
resultados_v = {}

# Iterar sobre cada variável de decisão
for v in prob_v.variables():
    if v.varValue != 0:
        # Extrair o nome da pessoa, o instrumento e o dia
        _, pessoa, instrumento, dia = v.name.split("_", 3)

        # Se o dia ainda não está no dicionário, adicione-o
        if dia not in resultados_v:
            resultados_v[dia] = {}

        # Adicione a pessoa e o instrumento ao dia correspondente
        resultados_v[dia][instrumento] = pessoa

df_v = pd.DataFrame(columns=V.columns, index=D.index)


# Preencher o DataFrame com os resultados_v
for dia in resultados_v:
    for instrumento in resultados_v[dia]:
        df_v.loc[dia, instrumento] = resultados_v[dia][instrumento]

# # Substituir NaN por ''
df_v = df_v.fillna('')

# # Convertendo os índices para o tipo datetime
df_v.index = pd.to_datetime(df_v.index, format='%d_%m_%Y')

# Formatando os índices para o formato desejado e criando um novo índice
df_v.index =  df_v.index.strftime('%d/%m/%Y')

df_v

Status: Optimal
Valor da função objetivo = 555.0


,Ministração,Soprano,Tenor,Contralto
Dias,,,,
14/01/2024,Ricardo,,,
21/01/2024,Henrique,,Ricardo,
28/01/2024,Henrique,,Marcela,
04/02/2024,Rafael,,,
11/02/2024,Henrique,,Ricardo,
18/02/2024,Henrique,Marcela,Ricardo,
25/02/2024,Rafael,,,
03/03/2024,Henrique,,Ricardo,
10/03/2024,Ricardo,,,Marcela


In [15]:
final = pd.concat([df,df_v], axis=1)

final

,Bateria,Baixo,Teclado,Violão,Guitarra,Ministração,Soprano,Tenor,Contralto
Dias,,,,,,,,,
14/01/2024,Lucas,Daniel,Bernardo,Ricardo,Osvaldo,Ricardo,,,
21/01/2024,Henrique,Ricardo,Mauri,Daniel,Osvaldo,Henrique,,Ricardo,
28/01/2024,Marcela,Gabriel,Vinicius,Henrique,Osvaldo,Henrique,,Marcela,
04/02/2024,Lucas,Rafael,Bernardo,Osvaldo,Enzo,Rafael,,,
11/02/2024,Henrique,Ricardo,Vinicius,Osvaldo,Daniel,Henrique,,Ricardo,
18/02/2024,Marcela,Gabriel,Henrique,Ricardo,Osvaldo,Henrique,Marcela,Ricardo,
25/02/2024,Lucas,Enzo,Vinicius,Rafael,Osvaldo,Rafael,,,
03/03/2024,Henrique,Daniel,Vinicius,Ricardo,Osvaldo,Henrique,,Ricardo,
10/03/2024,Marcela,Gabriel,Bernardo,Ricardo,Osvaldo,Ricardo,,,Marcela


In [14]:
disponibilidade_teste = pd.DataFrame(0, columns=P.index, index=D.index)

# # Convertendo os índices para o tipo datetime
disponibilidade_teste.index = pd.to_datetime(disponibilidade_teste.index, format='%d_%m_%Y')

# Formatando os índices para o formato desejado e criando um novo índice
disponibilidade_teste.index =  disponibilidade_teste.index.strftime('%d/%m/%Y')

for i in final.columns:
  for j in final.index:
    row_index = j
    column_name = final[i][j]
    disponibilidade_teste.loc[row_index, column_name] = 1
    
disponibilidade_teste

Pessoas,Daniel,Enzo,Gabriel,Henrique,Lucas,Marcela,Mauri,Osvaldo,Ricardo,Vinicius,Israel,Everlyn,Dani,Júlia,Rafael,Tassi,Bernardo,Ariane,
Dias,,,,,,,,,,,,,,,,,,,
14/01/2024,1,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,1,0,1.0
21/01/2024,1,0,0,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,1.0
28/01/2024,0,0,1,1,0,1,0,1,0,1,0,0,0,0,0,0,0,0,1.0
04/02/2024,0,1,0,0,1,0,0,1,0,0,0,0,0,0,1,0,1,0,1.0
11/02/2024,1,0,0,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,1.0
18/02/2024,0,0,1,1,0,1,0,1,1,0,0,0,0,0,0,0,0,0,1.0
25/02/2024,0,1,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,1.0
03/03/2024,1,0,0,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,1.0
10/03/2024,0,0,1,0,0,1,0,1,1,0,0,0,0,0,0,0,1,0,1.0
